# EMIT Hyperspectral Analysis: Cuprite, NV

This notebook demonstrates the analysis of NASA EMIT hyperspectral data for mineral identification in Cuprite, Nevada.

Target Coordinates: 37.52, -117.19

In [ ]:
# Install dependencies if not already installed
%pip install geemap earthengine-api

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import json
import sys

# Import the logic from extract_emit.py
from extract_emit import process_emit

In [ ]:
# Authenticate and Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Define target location
lat, lon = 37.52, -117.19

# Run the analysis
print(f"Processing EMIT data for {lat}, {lon}...")
output, images = process_emit(lat, lon)

if images is None:
    print("Error:", output.get('error'))
else:
    print("Analysis complete.")
    print("Spatial Mean Statistics:", json.dumps(output['spatial_mean'], indent=2))

In [ ]:
# Fetch a recent Sentinel-2 image for context (Visible Base Layer)
s2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(ee.Geometry.Point([lon, lat])) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

vis_s2 = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

In [ ]:
# Create Interactive Map
Map = geemap.Map(center=[lat, lon], zoom=13)

if images:
    # 1. Sentinel-2 Base (Visible)
    Map.addLayer(s2, vis_s2, "Sentinel-2 Base")

    # 2. NDVI Mask (Show where vegetation was removed)
    # We display the valid mask (Green = Analyzed/No Veg, Red = Masked/Vegetation)
    mask = images['mask']
    # Mask 0 is invalid (vegetation), 1 is valid (soil)
    # Let's visualize the mask itself.
    Map.addLayer(mask, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, "NDVI Mask (Green=Soil, Red=Veg)")

    # 3. Mineral Depth (Heatmap)
    # Select the 'corrected_depth' or 'depth' band
    depth = images['processed'].select('corrected_depth')
    Map.addLayer(depth, {'min': 0, 'max': 0.15, 'palette': ['blue', 'yellow', 'red']}, "Mineral Depth Heatmap")

    # Add a marker for the center
    Map.add_marker([lat, lon], title="Target Center")

Map

In [ ]:
# Find the pixel with the deepest feature and plot its spectral signature

if images:
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(6000).bounds()
    
    depth_img = images['processed'].select('corrected_depth')
    
    # Find maximum depth value in the region
    max_val = depth_img.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=region,
        scale=60,
        maxPixels=1e6
    ).get('corrected_depth')
    
    # Create a mask for the max value pixel(s)
    # Note: floating point comparison can be tricky, using a small tolerance or just equal if exact match
    max_mask = depth_img.gte(ee.Number(max_val).subtract(0.0001))
    
    # Get coordinates of the max pixel
    pixel_img = depth_img.addBands(ee.Image.pixelLonLat())
    
    max_loc = pixel_img.updateMask(max_mask).reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=region,
        scale=60,
        maxPixels=1e6
    )
    
    if max_loc.get('latitude') is not None:
        max_lat = max_loc.get('latitude')
        max_lon = max_loc.get('longitude')
        max_point = ee.Geometry.Point([max_lon, max_lat])
        
        print(f"Max depth ({max_val.getInfo():.4f}) found at: {max_lat.getInfo():.4f}, {max_lon.getInfo():.4f}")
        
        # Extract full spectrum from original image at this location
        full_spectrum_dict = images['original'].reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=max_point,
            scale=60
        ).getInfo()
        
        # Prepare data for plotting
        wavelengths = output['metadata']['wavelengths']
        band_names = output['metadata']['band_names']
        
        spectrum_data = []
        for i, band in enumerate(band_names):
            val = full_spectrum_dict.get(band)
            if val is not None:
                spectrum_data.append((wavelengths[i], val))
        
        # Sort by wavelength
        spectrum_data.sort(key=lambda x: x[0])
        wvl = [x[0] for x in spectrum_data]
        refl = [x[1] for x in spectrum_data]
        
        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(wvl, refl, label='Reflectance')
        
        # Highlight the SWIR range where we looked for features
        plt.axvspan(2000, 2500, color='yellow', alpha=0.2, label='SWIR Range (2000-2500nm)')
        
        plt.title(f"Spectral Signature at Max Depth Location")
        plt.xlabel("Wavelength (nm)")
        plt.ylabel("Reflectance")
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()
    else:
        print("Could not determine location of max depth.")
else:
    print("Skipping chart generation due to processing error.")